In [7]:
%%bash
pip install ../

Processing /Users/sebastientourbier/Softwares/forks/mialsuperresolutiontoolkit-1
  Created wheel for pymialsrtk: filename=pymialsrtk-2.0.2.dev0-py3-none-any.whl size=174085433 sha256=93ab64303a0db9741479e029747d49a7df32fef25967698698a4ab87028eed27
  Stored in directory: /Users/sebastientourbier/Library/Caches/pip/wheels/7e/e7/e7/f2999683d01f3a046157fafe8d3e57f99e2d0367c0e2d45e88
Successfully built pymialsrtk
  Attempting uninstall: pymialsrtk
    Found existing installation: pymialsrtk 2.0.2.dev0
    Uninstalling pymialsrtk-2.0.2.dev0:
      Successfully uninstalled pymialsrtk-2.0.2.dev0


In [1]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

In [2]:
from traits.api import *

import nibabel as nib

from nipype.utils.filemanip import split_filename
from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

from nipype.interfaces.mixins import reporting

In [3]:
import nibabel as nib
import numpy as np
import skimage.measure

In [4]:
bids_dir = os.path.join('/Users/sebastientourbier/Softwares/forks/mialsuperresolutiontoolkit-1/data')
mask_dir = os.path.join(bids_dir, 'derivatives', 'manual_masks')

subject = '01'

layout = BIDSLayout(bids_dir, derivatives=mask_dir)
print(layout)

/Applications/miniconda3/envs/pymialsrtk-env/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


BIDS Layout: ...lsuperresolutiontoolkit-1/data | Subjects: 1 | Sessions: 0 | Runs: 6


In [5]:
from pymialsrtk.interfaces.preprocess import StacksOrdering

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
stacksOrdering = StacksOrdering()

stacksOrdering.inputs.input_masks = [
    mask_dir + '/sub-01/anat/sub-01_run-1_T2w_desc-brain_mask.nii.gz',
    mask_dir + '/sub-01/anat/sub-01_run-2_T2w_desc-brain_mask.nii.gz',
    mask_dir + '/sub-01/anat/sub-01_run-3_T2w_desc-brain_mask.nii.gz',
    mask_dir + '/sub-01/anat/sub-01_run-4_T2w_desc-brain_mask.nii.gz',
    mask_dir + '/sub-01/anat/sub-01_run-5_T2w_desc-brain_mask.nii.gz',
    mask_dir + '/sub-01/anat/sub-01_run-6_T2w_desc-brain_mask.nii.gz'
]

aa = stacksOrdering.run() # doctest: +SKIP
aa.outputs.stacks_order

  Info: Number of NaNs = 0
  Info: Number of NaNs = 0
  Info: Number of NaNs = 0
  Info: Number of NaNs = 0
  Info: Number of NaNs = 0
  Info: Number of NaNs = 0
	>> Create report image...
			 - Format data...
			 - Create DataFrame...
     Scan  Slice  Motion Index  Proportion of NaNs (%)    X (mm)    Y (mm)  \
5   run-2  run-2      0.023707                     0.0  0.440528  0.248819   
6   run-2  run-2      0.023707                     0.0  0.143335 -0.194660   
7   run-2  run-2      0.023707                     0.0 -0.058334  0.358721   
8   run-2  run-2      0.023707                     0.0  0.445515 -0.582403   
9   run-2  run-2      0.023707                     0.0 -0.971044  0.169523   
24  run-5  run-5      0.034545                     0.0 -0.115317 -0.936588   
25  run-5  run-5      0.034545                     0.0 -0.691212  0.427981   
26  run-5  run-5      0.034545                     0.0  0.151605  0.099944   
27  run-5  run-5      0.034545                     0.0 -0.3668

[2, 5, 3, 4, 6, 1]

In [ ]:
#Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

print('Check for {}'.format(mialsrtk_dataset_description_json))
if not os.access(mialsrtk_dataset_description_json, os.R_OK):
    import json
    data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
                                'Version': 'v2.0.0-beta', 
                                'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
                                  },
            'Name': 'MIAL Super-Resolution ToolKit',
            'BIDSVersion': '1.2.0'
           }
    os.makedirs(output_dir)
    with open(mialsrtk_dataset_description_json, 'w+') as outfile:
        json.dump(data, outfile, indent=4)
    print('File {} was created'.format(mialsrtk_dataset_description_json))
else:
    print('File {} already exists'.format(mialsrtk_dataset_description_json))
    

wf = Workflow(name="mon_stacksOrdering",base_dir=os.path.join(output_dir,'sub-{}'.format(subject),'nipype'))

bg = Node(interface=BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.index_derivatives = True
bg.inputs.raise_on_empty = False
bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"]),
                          'masks': dict(suffix='mask',datatype='anat',extensions=[".nii",".nii.gz"])}

stacksOrdering = Node(interface = StacksOrdering(),name='mon_stacksOrdering_module')

wf.connect(bg, "masks", stacksOrdering, "input_masks")

res = wf.run()

In [ ]:
%%bash
pip install matplotlib==3.4